In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/ecospold2matrix/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pymrio/')
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/')
import ecospold2matrix as e2m
import pymrio
import pylcaio

import os
import gzip
import pickle
import pandas as pd
import itertools

# from importlib import reload
# import pdb
import scipy.sparse

## Prepare Ecoinvent data with ecospold2matrix

In [ ]:
# %%time

# #run ecospold2matrix
# ecospold_dir = '/home/jakobs/Documents/IndEcol/Data/ecoinvent/ecoinvent 3.5_cutoff_ecoSpold02/'
# project_name = 'ecoinvent_35_cutoff'
# out_dir = '/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/'

# parser = e2m.Ecospold2Matrix(ecospold_dir, project_name, out_dir, positive_waste=True, nan2null=True,
#                             characterisation_file=
#         '/home/jakobs/data/ecoinvent/ecoinvent 3.5_LCIA_implementation/LCIA_implementation_3.5.xlsx', )
# parser.ecospold_to_Leontief(fileformats='Pandas',with_absolute_flows=False)

## Load ecoinvent pickle files

In [2]:
%%time
with gzip.open('/home/jakobs/Documents/IndEcol/Data/ecospold2matrix_matrices/ecoinvent_35_cutoffPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

CPU times: user 7.78 s, sys: 440 ms, total: 8.22 s
Wall time: 8.23 s


### Imput missing ecoinvent price data
Ecospold2matrix seems to have a bug and not extract all price data. However, the datasheets in filter.xlsx contains the actual ecoinvent prices for the processes so manually imput them into the PRO dataframe

In [3]:
file =  pd.ExcelFile('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/Data/eco3.5_exio3/Filter.xlsx')
sheetnames = file.sheet_names

for sheet_name in sheetnames:
    if not sheet_name == 'No price':
        print(sheet_name)
        sheet = file.parse(sheet_name, index_col=0)
        #find UUID of those processes that do not have a price (NaN)
        indices = list(set(ecoinvent['PRO'].loc[ecoinvent['PRO'].price.isnull()].index.tolist()).intersection(
                  sheet.index.tolist()))
        ecoinvent['PRO'].loc[indices, 'price'] = sheet.loc[indices, 'Price']
        ecoinvent['PRO'].loc[indices, 'priceUnit'] = 'EUR2005'

Hybridized
Market
Not commercialized
Poor quality
Empty processes
Uncovered geography


## Load Exiobase into a Dataframe with pymrio

In [4]:
%%time
EXIOBASE_path = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase3_6/IOT_2012_pxp'
io = pymrio.parse_exiobase3(EXIOBASE_path)

CPU times: user 35 s, sys: 924 ms, total: 36 s
Wall time: 36.2 s


In [ ]:
# path_to_capital_matrix = '/home/jakobs/Documents/IndEcol/Data/EXIOBASE/exiobase3_6/capital consumption matrices/Kbar_exio_v3_6_2012pxp.mat'

## Load aggregation dictionary

In [ ]:
# agg_dic = Aggregation_dic_from_file(os.path.join(os.path.dirname(os.path.dirname(EXIOBASE_path)),'Product_Country_aggregation.xlsx'))

## Read in data with pylcaio

In [5]:
%%time
database_loader = pylcaio.DatabaseLoader(ecoinvent, io, 'ecoinvent3.5', 'exiobase3')

CPU times: user 11.8 ms, sys: 124 µs, total: 11.9 ms
Wall time: 5.79 ms


In [6]:
%%time
lcaio_object = database_loader.combine_ecoinvent_exiobase(complete_extensions=False, 
                                        impact_world=False, regionalized=False)

No path for the capital folder was provided. Capitals will not be endogenized
CPU times: user 2min 49s, sys: 5.39 s, total: 2min 54s
Wall time: 1min 47s


In [ ]:
# lcaio_object.aggregate(agg_dic=agg_dic, build_aggregation_matrix_function=build_MultiIndex_Aggregation_Matrix)

In [7]:
%%time
lcaio_object.hybridize(price_neutral_cut_off_matrix=False, capitals=False, priceless_scaling=False, calculate_cutoff_variance=False)

Indentifying Rest of World regions...
Updating electricity prices...
Calculating productions volumes...
Adjusting low production volume processes...
Extending inventory...
Building H matrix...
Building geography concordance...
Building geographic conconcordance took 4.729621171951294 seconds
Filter H matrix...
Build Cut-off matrix...
CPU times: user 2min 16s, sys: 11.6 s, total: 2min 28s
Wall time: 2min 24s


In [8]:
%%time
# lcaio_object.variance_flag = False
lcaio_object.correct_double_counting('STAM', capitals=False)

STAM double counting correction being appliedi...
... Calculating Lambda Filter Matrix...
... Calculating Gamma Filter Matrix...
... Calculating Phi Filter Matrix...
Correcting cut-off matrices...
CPU times: user 7min 17s, sys: 13.4 s, total: 7min 31s
Wall time: 5min 20s


In [10]:
%%time
file_name_path = '/home/jakobs/Documents/IndEcol/OASES/Accuracy_vs_precission_HLCA/Databases/plcaio_object_STAM.pickle'
with gzip.open(file_name_path, 'wb') as fh:
    pickle.dump(lcaio_object, fh)
print('lcaio_object written to" {}'.format(file_name_path))

lcaio_object written to" /home/jakobs/Documents/IndEcol/OASES/Accuracy_vs_precission_HLCA/Databases/plcaio_object_STAM.pickle
CPU times: user 4min 58s, sys: 2.42 s, total: 5min
Wall time: 5min


In [ ]:
# %%time
# lcaio_object.correct_double_counting('binary', capitals=False)

In [9]:
%%time
lcaio_object.save_system(file_path='/home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/', file_name='extend_inventory_fixed-absolute-noise-nulling_2012_STAM', format='pickle')

Database saved to /home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/extend_inventory_fixed-absolute-noise-nulling_2012_STAM
Description file saved to /home/jakobs/Documents/IndEcol/OASES/TEST_PYLCAIO/Databases/description_extend_inventory_fixed-absolute-noise-nulling_2012_STAM.txt
CPU times: user 22.3 s, sys: 316 ms, total: 22.6 s
Wall time: 22.5 s
